## Generative Adversarial Networks

In [ ]:
import os
from tqdm import tqdm
import numpy as np
from PIL import Image
import tensorflow as tf
import matplotlib.pyplot as plt

In [ ]:
def load_img(filename):
    return np.asarray(Image.open(filename).convert('RGB'))

In [ ]:
def load_faces(dir_, n_faces):
    faces = list()
    
    for filename in os.listdir(dir_):
        filename = os.path.join(dir_, filename)
        pixels = load_img(filename)
        faces.append(pixels)
        
        if len(faces) >= n_faces:
            break
        
    return np.asarray(faces)

In [ ]:
def plot_faces(faces, n):
    plt.figure(figsize=(15, 10))
    for i in range(n**2):
        plt.subplot(n, n, 1+i)
        plt.axis('off')
        plt.imshow(faces[i])
    plt.show()

In [ ]:
DATA_DIR = '../../Resources/data/gan/celeba-dataset/img_align_celeba/img_align_celeba/'
faces = load_faces(DATA_DIR, 25)
faces.shape

In [ ]:
plot_faces(faces, 5)

In [ ]:
import mtcnn
mtcnn.__version__

In [ ]:
def extract_faces(model, pixels, required_size=(80, 80)):
    faces = model.detect_faces(pixels)
    
    if len(faces) == 0:
        return None
    
    x1, y1, width, height = faces[0]['box']
    x1, y1 = abs(x1), abs(y1)
    
    x2, y2 = x1 + width, y1 + height
    
    face_pixels = pixels[y1:y2, x1:x2]
    
    image = Image.fromarray(face_pixels)
    image = image.resize(required_size)
    face_array = np.asarray(image)
    
    return face_array

In [ ]:
## Updating load_faces

from tqdm import tqdm

def load_faces(dir_, n_faces):
    model = mtcnn.MTCNN()
    faces = list()
    
    for filename in tqdm(os.listdir(dir_)):
        filename = os.path.join(dir_, filename)
        pixels = load_img(filename)
        face = extract_faces(model, pixels)
        
        if face is None:
            continue
            
        faces.append(face)
        
        if len(faces) >= n_faces:
            break
            
    return np.asarray(faces)

In [ ]:
faces = load_faces(DATA_DIR, 50)
faces.shape

In [ ]:
np.savez_compressed(os.path.join(DATA_DIR, 'compressed_faces.npz'), faces)

## GAN

In [ ]:
def make_discriminator(in_shape=(80, 80, 3)):
    model = tf.keras.models.Sequential()
    
    ## Conv
    model.add(tf.keras.layers.Conv2D(128, (5, 5), padding='same', input_shape=in_shape))
    model.add(tf.keras.layers.LeakyReLU(alpha=0.2))
    
    ## Downsample
    model.add(tf.keras.layers.Conv2D(128, (5, 5), strides=(2, 2), padding='same'))
    model.add(tf.keras.layers.LeakyReLU(alpha=0.2))
    
    ## Downsample
    model.add(tf.keras.layers.Conv2D(128, (5, 5), strides=(2, 2), padding='same'))
    model.add(tf.keras.layers.LeakyReLU(alpha=0.2))
    
    ## Downsample
    model.add(tf.keras.layers.Conv2D(128, (5, 5), strides=(2, 2), padding='same'))
    model.add(tf.keras.layers.LeakyReLU(alpha=0.2))
    
    ## Downsample
    model.add(tf.keras.layers.Conv2D(128, (5, 5), strides=(2, 2), padding='same'))
    model.add(tf.keras.layers.LeakyReLU(alpha=0.2))
    
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dropout(0.4))
    model.add(tf.keras.layers.Dense(1, activation='sigmoid'))
    
    opt = tf.keras.optimizers.Adam(lr=0.0002, beta_1=0.5)
    
    model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
    
    return model

In [ ]:
def make_generator(latent_dim):
    
    model = tf.keras.models.Sequential()

    n_nodes = 128 * 5 * 5
    
    
    model.add(tf.keras.layers.Dense(n_nodes, input_dim=latent_dim))
    model.add(tf.keras.layers.LeakyReLU(alpha=0.2))
    model.add(tf.keras.layers.Reshape((5, 5, 128)))
    
    
    # upsample to 10x10
    model.add(tf.keras.layers.Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
    model.add(tf.keras.layers.LeakyReLU(alpha=0.2))
    
    
    # upsample to 20x20
    model.add(tf.keras.layers.Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
    model.add(tf.keras.layers.LeakyReLU(alpha=0.2))
    
    
    # upsample to 40x40
    model.add(tf.keras.layers.Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
    model.add(tf.keras.layers.LeakyReLU(alpha=0.2))
    
    
    # upsample to 80x80
    model.add(tf.keras.layers.Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
    model.add(tf.keras.layers.LeakyReLU(alpha=0.2))
    
    
    # output layer 80x80x3
    model.add(tf.keras.layers.Conv2D(3, (5,5), activation='tanh', padding='same'))
    
    
    return model

In [ ]:
def make_gan(g_model, d_model):
    
    d_model.trainable = False
    
    
    model = tf.keras.models.Sequential()

    model.add(g_model)
    model.add(d_model)

    
    opt = tf.keras.optimizers.Adam(lr=0.0002, beta_1=0.5)
    model.compile(loss='binary_crossentropy', optimizer=opt)
    
    return model

In [ ]:
def load_real_samples(DATA_DIR):
    # load the face dataset
    data = np.load(os.path.join(DATA_DIR, 'compressed_faces.npz'))
    X = data['arr_0']
    # convert from unsigned ints to floats
    X = X.astype('float32')
    # scale from [0,255] to [-1,1]
    X = (X - 127.5) / 127.5
    return X

In [ ]:
def generate_real_samples(dataset, n_samples):

    ix = np.random.randint(0, dataset.shape[0], n_samples)
    X = dataset[ix]
    y = np.ones((n_samples, 1))
    return X, y

In [ ]:
def generate_latent_points(latent_dim, n_samples):
    x_input = np.random.randn(latent_dim * n_samples)
    # reshape into a batch of inputs for the network
    x_input = x_input.reshape(n_samples, latent_dim)
    return x_input

In [ ]:
def generate_fake_samples(g_model, latent_dim, n_samples):
    # generate points in latent space
    x_input = generate_latent_points(latent_dim, n_samples)
    
    # predict outputs
    X = g_model.predict(x_input)
    
    # create 'fake' class labels (0)
    y = np.zeros((n_samples, 1))
    return X, y

In [ ]:
def save_plot(examples, epoch, n=10):
    # scale from [-1,1] to [0,1]
    examples = (examples + 1) / 2.0
    # plot images
    for i in range(n * n):
        # define subplot
        plt.subplot(n, n, 1 + i)
        # turn off axis
        plt.axis('off')
        # plot raw pixel data
        plt.imshow(examples[i])
    # save plot to file
    filename = 'generations/vanilla_gan/generated_plot_e%03d.png' % (epoch+1)
    plt.savefig(filename)
    plt.close()

In [ ]:
def summarize_performance(epoch, g_model, d_model, dataset, latent_dim, n_samples=100):
    # prepare real samples
    X_real, y_real = generate_real_samples(dataset, n_samples)
    # evaluate discriminator on real examples
    _, acc_real = d_model.evaluate(X_real, y_real, verbose=0)
    # prepare fake examples
    x_fake, y_fake = generate_fake_samples(g_model, latent_dim, n_samples)
    # evaluate discriminator on fake examples
    _, acc_fake = d_model.evaluate(x_fake, y_fake, verbose=0)
    # summarize discriminator performance
    print('* Accuracy real: %.0f%%, fake: %.0f%%' % (acc_real*100, acc_fake*100))
    # save plot
    save_plot(x_fake, epoch)
    # save the generator model tile file
    filename = '../../Resources/models/gan/vanilla_generator_model_%03d.h5' % (epoch+1)
    g_model.save(filename)

In [ ]:
def train(generator, discriminator, gan_model, dataset, latent_dim, n_epochs=100, n_batch=32):
    
    batch_per_epoch = dataset.shape[0] // n_batch
    half_batch = batch_per_epoch // 2
    
    print('Batch per epoch: ', batch_per_epoch)
    
    for i in range(n_epochs):
        for j in range(batch_per_epoch):
            
            
            X_real, y_real = generate_real_samples(dataset, half_batch)
            d_loss1, _ = discriminator.train_on_batch(X_real, y_real)
            
            X_fake, y_fake = generate_fake_samples(generator, latent_dim, half_batch)
            d_loss2, _ = discriminator.train_on_batch(X_fake, y_fake)
            
            ## Prepare points in latent space as inputs for generator
            X_gan = generate_latent_points(latent_dim, n_batch)
            y_gan = np.ones((n_batch, 1))
            
            ## Update the generator via discriminator's error
            g_loss = gan_model.train_on_batch(X_gan, y_gan)
            
            print('* Epoch: {}, Batch: {}, d1: {}, d2: {}, g: {}'.format(i+1, j+1, batch_per_epoch, d_loss1, d_loss2, g_loss))
            
        if (i + 1) % 10 == 0:
            summarize_performance(i, generator, discriminator, dataset, latent_dim)

In [ ]:
latent_dim = 100


discriminator = make_discriminator()
generator = make_generator(latent_dim)
gan = make_gan(generator, discriminator)

DATA_DIR = '../../Resources/data/gan/celeba-dataset/img_align_celeba/img_align_celeba/'
dataset = load_real_samples(DATA_DIR)


train(generator, discriminator, gan, dataset, latent_dim)